<a href="https://colab.research.google.com/github/mickeykubo/fastai-v1/blob/master/lesson4_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular models

In [1]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    737      0 --:--:-- --:--:-- --:--:--   737
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [0]:
from fastai.tabular import *

問題

http://logopt.com/data/Boston.csv のBostonの住宅データを用いて回帰分析を行え．

medvが住宅の価格で，他のデータ（犯罪率や人口など）から予測する．

In [28]:
boston = pd.read_csv('http://logopt.com/data/Boston.csv',index_col=0)
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [0]:
procs = [FillMissing, Categorify, Normalize] #前処理の種類を準備．
#dep_var = ['medv', "lstat"]  #従属変数名とカテゴリー変数が格納されている列リストを準備．
dep_var = 'medv'  #従属変数名とカテゴリー変数が格納されている列リストを準備．
cat_names = []
cont_names = boston.columns

In [0]:
start, finish = int(len(boston)*0.7), len(boston)

In [0]:
test = TabularList.from_df(boston.iloc[start:finish].copy(),  cat_names=cat_names, cont_names=cont_names, )

In [0]:
data = (TabularList.from_df(boston, cat_names=cat_names,  cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(start,finish)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [40]:
data.show_batch(rows=10)

crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,target
-0.4231,-0.6069,0.0296,-0.2869,0.0989,0.1040,1.2857,-0.9942,0.3282,1.0794,1.4573,0.3657,0.3232,-0.6229,19.8
-0.5336,-0.6069,-0.6383,-0.2869,-0.5956,0.3353,-0.1569,-0.0197,-0.9145,-0.9343,0.3612,0.2927,-0.6531,0.1830,26.6
-0.5000,2.4679,-0.5628,-0.2869,-0.9673,1.1034,-1.1561,0.3104,-0.2931,-0.9637,0.6809,0.4069,-1.1582,1.4512,37.3
-0.4645,-0.6069,0.7304,-0.2869,-0.7130,-0.1851,0.4905,-0.2071,0.3282,1.2852,0.4526,-0.1699,0.2722,-0.5992,20.0
-0.5801,2.2757,-0.8910,-0.2869,-0.8010,0.2835,-1.3637,0.4487,-0.9145,-0.8608,0.2699,0.3753,-1.0290,0.6808,30.8
-0.0635,-0.6069,-0.3577,3.4752,-0.0283,0.8113,0.9831,-0.7858,2.1924,-0.0524,-0.1411,0.2778,-0.1122,0.1949,26.7
-0.0744,-0.6069,-0.1640,-0.2869,-0.1652,-0.5380,-0.3750,0.1181,0.3282,-0.3464,0.8636,0.4069,-0.4541,-0.5518,20.4
-0.3556,-0.6069,0.0296,-0.2869,0.0989,-0.8420,1.1027,-1.1021,0.3282,1.0794,1.4573,0.3763,1.4101,-0.6585,19.5
3.6501,-0.6069,1.8381,-0.2869,3.5322,-1.1964,1.2083,-1.4352,0.3282,1.3587,-1.3742,-0.7112,1.8846,-1.1444,15.4
0.3739,-0.6069,-0.0393,-0.2869,0.2750,-0.4565,0.8424,-0.0079,-0.2931,-0.0524,1.5029,-0.0121,-0.0187,-0.8126,18.2


In [0]:
learn = tabular_learner(data, layers=[200,100])

In [42]:
learn.fit(10, 1e-2)

epoch,train_loss,valid_loss
1,665.836304,357.416534
2,641.892578,406.800140
3,602.898376,573.235657
4,564.476074,391.435852
5,522.182373,292.670441
6,472.507629,222.037750
7,418.220947,487.307159
8,361.651520,305.371826
9,308.509613,19.361244
10,262.615143,8.382178


線形回帰との比較

In [0]:
X = boston.iloc[:,range(13)] # 最後の列以外のデータを独立変数（特徴ベクトル）として抽出
y = boston.medv #最後の列(medv)を従属変数として抽出

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)#30%のデータをテスト用，それ以外を学習用に分離

In [16]:
from sklearn.linear_model import LinearRegression #クラスのインポート
lin_reg = LinearRegression()     # 線形回帰クラスのインスタンス生成
lin_reg.fit(X_train, y_train)    # 学習
yhat = lin_reg.predict(X_test)   # 予測
print(lin_reg.score(X_test,y_test)) #決定係数

0.7203016574262371


In [0]:
from sklearn import metrics

In [18]:
metrics.mean_squared_error(y_test,yhat)

24.192453905781946

サラリーが閾値より高いかどうかの判別の例題

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [0]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [0]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,7th-8th,Never-married,Other-service,Own-child,White,False,0.0303,0.0433,-2.3781,<50k
Private,12th,Married-civ-spouse,Adm-clerical,Husband,White,False,0.4701,1.6038,-0.8135,<50k
Private,HS-grad,Never-married,Other-service,Other-relative,Asian-Pac-Islander,False,-0.9959,-0.3771,-0.4224,<50k
Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,False,-0.5561,-1.3820,1.1422,<50k
Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.1896,0.1254,1.1422,<50k
Private,Some-college,Separated,Exec-managerial,Unmarried,White,False,0.1036,0.0559,-0.0312,<50k
Private,Assoc-voc,Never-married,Exec-managerial,Own-child,White,False,0.5434,0.0141,0.3599,<50k
Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,False,-0.7027,-0.3635,-0.4224,<50k
Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,False,0.3235,-1.0738,-0.4224,<50k
Self-emp-not-inc,HS-grad,Married-civ-spouse,Sales,Husband,White,False,-0.6294,0.5114,-0.4224,<50k


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [0]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.368374,0.378342,0.820000


## Inference

In [0]:
row = df.iloc[0]

In [0]:
learn.predict(row)

(Category >=50k, tensor(1), tensor([0.3054, 0.6946]))

ロジスティック回帰との比較

In [0]:
df2 = pd.get_dummies(df, drop_first=True) #ダミー変数の最初のものを除く
df2.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,salary_>=50k
0,49,101320,12.0,0,1902,40,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,44,236746,14.0,10520,0,45,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2,38,96185,NaN,0,0,32,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,38,112847,15.0,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,42,82297,NaN,0,0,50,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [0]:
y = df2["salary_>=50k"]             # 従属変数の抽出
X = df2.drop('salary_>=50k',axis=1) # 独立変数（特徴ベクトル）
X.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,49,101320,12.0,0,1902,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,44,236746,14.0,10520,0,45,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,38,96185,NaN,0,0,32,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,38,112847,15.0,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,42,82297,NaN,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [0]:
X = X.dropna(axis=1)

In [0]:
from sklearn.linear_model import LogisticRegression #線形回帰クラスのインポート
reg = LogisticRegression() # 線形回帰クラスのインスタンス生成
reg.fit(X, y)           # 学習
yhat = reg.predict(X)   # 予測

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
from sklearn import metrics
print(metrics.accuracy_score(y, yhat))

0.7978563311937594
